In [4]:
import sys
from config import *
import pandas as pd

In [2]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [19]:
id_list=[]
def get_id(title):
    
    for song in title:
        try:
            results = sp.search(q="track:"+song,limit=1) 
            song_id=results['tracks']['items'][0]['id']
            id_list.append(song_id)
            #print("Checking on song {}".format(song))
        except IndexError:
            song_id=''
            id_list.append(song_id)   

In [12]:
hot100=pd.read_csv('hot100.csv',index_col=0)

In [8]:
nothot=pd.read_csv('charts.csv')

In [9]:
nothot=nothot[['song','artist']]

In [10]:
nothot=nothot.drop_duplicates()
nothot=nothot[0:3000]
nothot

,song,artist
0,Easy On Me,Adele
1,Stay,The Kid LAROI & Justin Bieber
2,Industry Baby,Lil Nas X & Jack Harlow
3,Fancy Like,Walker Hayes
4,Bad Habits,Ed Sheeran
...,...,...
26093,CRZY,Kehlani
26095,Lockjaw,French Montana Featuring Kodak Black
26096,You & Me,Marc E. Bassy Featuring G-Eazy
26146,For Free,DJ Khaled Featuring Drake


In [13]:
nothot['duplicated'] = 0 # 0 means not duplicated, 1 mean duplicated
for row_hot in range(len(hot100)): # range(len(hot)
    hot_song = hot100.iloc[row_hot,0]
    hot_artist = hot100.iloc[row_hot,1]
    print("Checking if song {} is duplicated".format(row_hot))
    for row_not_hot in range(len(nothot)):
        not_hot_song = nothot.iloc[row_not_hot,0]
        not_hot_artist = nothot.iloc[row_not_hot,1]
        if (( not_hot_song == hot_song ) and (not_hot_artist == hot_artist)):
            nothot.iloc[row_not_hot,2] = 1
            break # Exits the innermost for loop. Because if the song is found we don't need to continue the search.
            
nothot = nothot[nothot['duplicated'] == 0]
nothot = nothot.drop(columns="duplicated",axis=1).reset_index(drop=True)

Checking if song 0 is duplicated
Checking if song 1 is duplicated
Checking if song 2 is duplicated
Checking if song 3 is duplicated
Checking if song 4 is duplicated
Checking if song 5 is duplicated
Checking if song 6 is duplicated
Checking if song 7 is duplicated
Checking if song 8 is duplicated
Checking if song 9 is duplicated
Checking if song 10 is duplicated
Checking if song 11 is duplicated
Checking if song 12 is duplicated
Checking if song 13 is duplicated
Checking if song 14 is duplicated
Checking if song 15 is duplicated
Checking if song 16 is duplicated
Checking if song 17 is duplicated
Checking if song 18 is duplicated
Checking if song 19 is duplicated
Checking if song 20 is duplicated
Checking if song 21 is duplicated
Checking if song 22 is duplicated
Checking if song 23 is duplicated
Checking if song 24 is duplicated
Checking if song 25 is duplicated
Checking if song 26 is duplicated
Checking if song 27 is duplicated
Checking if song 28 is duplicated
Checking if song 29 is d

In [16]:
nothot.drop_duplicates(subset=['song'],keep=False, inplace=True)

In [17]:
nothot.value_counts()

song                artist                                       
'Til You Can't      Cody Johnson                                     1
Pop Out             Polo G Featuring Lil Tjay                        1
Play That Song      Train                                            1
Playing Games       Summer Walker                                    1
Please              Lil Baby & Lil Durk                              1
                                                                    ..
Green Light         Lorde                                            1
Greenlight          Pitbull Featuring Flo Rida & LunchMoney Lewis    1
Growing Pains       Alessia Cara                                     1
Guardian Angel      XXXTENTACION                                     1
whoa (mind in awe)  XXXTENTACION                                     1
Length: 2735, dtype: int64

In [20]:
nothot[['song']].apply(get_id)

song    None
dtype: object

In [23]:
nothot['id']=id_list

In [25]:
nothot=nothot.drop(nothot[nothot['id']==''].index)

In [45]:
nothot['id'].value_counts()

0gplL1WMoJ6iYaPgMCL0gX    1
2374M0fQpWi3dLnB54qaLX    1
2qT1uLXPVPzGgFOx4jtEuo    1
2OsDoAzo5SGq10qwmKSQBv    1
2vs5ubqg8QUbfyWZpMfvWe    1
                         ..
7szuecWAPwGoV1e5vGu8tl    1
5lH14PEjAXYuMTz0Dd4L1D    1
7riuy1t0Kt3SRgMtFxX0FR    1
1orjH4IkSH9VrWmRCToKV8    1
4tCtwWceOPWzenK2HAIJSb    1
Name: id, Length: 2442, dtype: int64

In [43]:
nothot.drop_duplicates(subset=['id'],keep=False, inplace=True)#to drop the songs with duplicate id

In [31]:
from tqdm.notebook import tqdm

In [32]:
def chunks(lst,n):
    """
    Yields n-sized chunks of lst
    """
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [46]:
audio_feats = []
for chunk in tqdm(list(chunks(nothot['id'],50))):
    try:
        audio_feats.append(sp.audio_features(chunk)) 
    except KeyboardInterrupt:
        break
    except:
        continue

  0%|          | 0/49 [00:00<?, ?it/s]

In [47]:
audio_feats2 = pd.Series([y for x in audio_feats for y in x], name='features')

In [48]:
df_audio=pd.DataFrame(audio_feats2)

In [49]:
df_audio.isna().sum()

features    3
dtype: int64

In [40]:
df_audio

,features
0,"{'danceability': 0.604, 'energy': 0.366, 'key'..."
1,"{'danceability': 0.736, 'energy': 0.704, 'key'..."
2,"{'danceability': 0.647, 'energy': 0.765, 'key'..."
3,"{'danceability': 0.807, 'energy': 0.893, 'key'..."
4,"{'danceability': 0.664, 'energy': 0.609, 'key'..."
...,...
2437,"{'danceability': 0.925, 'energy': 0.355, 'key'..."
2438,"{'danceability': 0.827, 'energy': 0.702, 'key'..."
2439,"{'danceability': 0.615, 'energy': 0.648, 'key'..."
2440,"{'danceability': 0.916, 'energy': 0.571, 'key'..."


In [50]:
df_audio[df_audio['features'].isna()].index.tolist()

[246, 657, 940]

In [52]:
nothot.drop([246, 657, 940], axis=0, inplace=True)

In [55]:
feat_df = pd.DataFrame(list(audio_feats2.dropna()))

In [56]:
feat_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.604,0.366,5,-7.519,1,0.0282,0.5780,0.000000,0.1330,0.130,141.981,audio_features,0gplL1WMoJ6iYaPgMCL0gX,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,https://api.spotify.com/v1/tracks/0gplL1WMoJ6i...,https://api.spotify.com/v1/audio-analysis/0gpl...,224695,4
1,0.736,0.704,3,-7.409,0,0.0615,0.0203,0.000000,0.0501,0.894,149.995,audio_features,27NovPIUIRrOZoCHxABJwK,spotify:track:27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,212000,4
2,0.647,0.765,1,-6.459,1,0.0600,0.1110,0.000000,0.3150,0.855,79.994,audio_features,58UKC45GPNTflCN6nwCUeF,spotify:track:58UKC45GPNTflCN6nwCUeF,https://api.spotify.com/v1/tracks/58UKC45GPNTf...,https://api.spotify.com/v1/audio-analysis/58UK...,161853,4
3,0.807,0.893,11,-3.745,0,0.0347,0.0451,0.000028,0.3660,0.537,126.011,audio_features,3rmo8F54jFF8OgYsqTxm5d,spotify:track:3rmo8F54jFF8OgYsqTxm5d,https://api.spotify.com/v1/tracks/3rmo8F54jFF8...,https://api.spotify.com/v1/audio-analysis/3rmo...,230747,4
4,0.664,0.609,1,-6.509,1,0.0707,0.3040,0.000000,0.0926,0.194,130.041,audio_features,3Vi5XqYrmQgOYBajMWSvCi,spotify:track:3Vi5XqYrmQgOYBajMWSvCi,https://api.spotify.com/v1/tracks/3Vi5XqYrmQgO...,https://api.spotify.com/v1/audio-analysis/3Vi5...,210560,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2434,0.925,0.355,0,-9.309,0,0.5500,0.0413,0.000000,0.1410,0.383,92.988,audio_features,4ezAjoV01phDz1SqO6SzEU,spotify:track:4ezAjoV01phDz1SqO6SzEU,https://api.spotify.com/v1/tracks/4ezAjoV01phD...,https://api.spotify.com/v1/audio-analysis/4ezA...,196587,4
2435,0.827,0.702,6,-7.264,1,0.2670,0.2260,0.000050,0.1580,0.694,130.020,audio_features,6Dwtha2FtZFoMEBh5GR2sq,spotify:track:6Dwtha2FtZFoMEBh5GR2sq,https://api.spotify.com/v1/tracks/6Dwtha2FtZFo...,https://api.spotify.com/v1/audio-analysis/6Dwt...,191720,4
2436,0.615,0.648,5,-3.792,0,0.2200,0.0411,0.000000,0.2770,0.260,169.912,audio_features,7iaw359G2XT14uTfV9feip,spotify:track:7iaw359G2XT14uTfV9feip,https://api.spotify.com/v1/tracks/7iaw359G2XT1...,https://api.spotify.com/v1/audio-analysis/7iaw...,223147,4
2437,0.916,0.571,10,-8.241,0,0.2000,0.0173,0.000001,0.1530,0.498,103.521,audio_features,5TqBAOIBe0bQA0spNdcG3k,spotify:track:5TqBAOIBe0bQA0spNdcG3k,https://api.spotify.com/v1/tracks/5TqBAOIBe0bQ...,https://api.spotify.com/v1/audio-analysis/5TqB...,183695,4


In [66]:
id_list=[]
def get_id(title):
    
    for song in title:
        try:
            results = sp.search(q="track:"+song,limit=1) 
            song_id=results['tracks']['items'][0]['id']
            id_list.append(song_id)
            #print("Checking on song {}".format(song))
        except IndexError:
            song_id=''
            id_list.append(song_id)  

In [67]:
hot100[['song']].apply(get_id)

song    None
dtype: object

In [68]:
len(id_list)

100

In [69]:
my_audio_feature_hot= sp.audio_features(id_list) 
# return the audio feature for hot songs on DF

df_hot_songs = pd.DataFrame(my_audio_feature_hot)
df_hot_songs.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.950,0.891,2,-2.653,1,0.2410,0.0645,0.000018,0.3090,0.912,133.010,audio_features,4C6Uex2ILwJi9sZXRdmqXp,spotify:track:4C6Uex2ILwJi9sZXRdmqXp,https://api.spotify.com/v1/tracks/4C6Uex2ILwJi...,https://api.spotify.com/v1/audio-analysis/4C6U...,170977,4
1,0.520,0.731,6,-5.338,0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
2,0.836,0.743,10,-6.305,0,0.0656,0.0995,0.000000,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
3,0.687,0.887,1,-5.040,0,0.0826,0.0575,0.000002,0.2700,0.853,114.941,audio_features,2KukL7UlQ8TdvpaA7bY3ZJ,spotify:track:2KukL7UlQ8TdvpaA7bY3ZJ,https://api.spotify.com/v1/tracks/2KukL7UlQ8Td...,https://api.spotify.com/v1/audio-analysis/2Kuk...,278282,4
4,0.625,0.533,10,-11.903,0,0.0596,0.6590,0.002660,0.0546,0.139,108.296,audio_features,29d0nY7TzCoi22XBqDQkiP,spotify:track:29d0nY7TzCoi22XBqDQkiP,https://api.spotify.com/v1/tracks/29d0nY7TzCoi...,https://api.spotify.com/v1/audio-analysis/29d0...,300840,4
5,0.686,0.507,1,-7.097,1,0.0357,0.6260,0.000075,0.3830,0.690,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
6,0.768,0.714,10,-5.110,1,0.0401,0.3520,0.000000,0.1500,0.842,131.443,audio_features,4h4QlmocP3IuwYEj2j14p8,spotify:track:4h4QlmocP3IuwYEj2j14p8,https://api.spotify.com/v1/tracks/4h4QlmocP3Iu...,https://api.spotify.com/v1/audio-analysis/4h4Q...,163026,4
7,0.463,0.642,1,-4.474,1,0.3400,0.3140,0.000000,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
8,0.911,0.712,1,-5.105,0,0.0817,0.0901,0.000027,0.0933,0.425,92.005,audio_features,6Sq7ltF9Qa7SNFBsV5Cogx,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4
9,0.733,0.670,5,-6.009,1,0.0751,0.1210,0.000000,0.1210,0.472,100.964,audio_features,0O6u0VJ46W86TxN9wgyqDj,spotify:track:0O6u0VJ46W86TxN9wgyqDj,https://api.spotify.com/v1/tracks/0O6u0VJ46W86...,https://api.spotify.com/v1/audio-analysis/0O6u...,192841,4


In [70]:
hot_all=pd.concat([hot100, df_hot_songs],axis=1)

In [71]:
hot_all

,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Super Freaky Girl,Nicki Minaj,0.950,0.891,2,-2.653,1,0.2410,0.0645,0.000018,0.3090,0.912,133.010,audio_features,4C6Uex2ILwJi9sZXRdmqXp,spotify:track:4C6Uex2ILwJi9sZXRdmqXp,https://api.spotify.com/v1/tracks/4C6Uex2ILwJi...,https://api.spotify.com/v1/audio-analysis/4C6U...,170977,4
1,As It Was,Harry Styles,0.520,0.731,6,-5.338,0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
2,About Damn Time,Lizzo,0.836,0.743,10,-6.305,0,0.0656,0.0995,0.000000,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
3,Break My Soul,Beyonce,0.687,0.887,1,-5.040,0,0.0826,0.0575,0.000002,0.2700,0.853,114.941,audio_features,2KukL7UlQ8TdvpaA7bY3ZJ,spotify:track:2KukL7UlQ8TdvpaA7bY3ZJ,https://api.spotify.com/v1/tracks/2KukL7UlQ8Td...,https://api.spotify.com/v1/audio-analysis/2Kuk...,278282,4
4,Running Up That Hill (A Deal With God),Kate Bush,0.625,0.533,10,-11.903,0,0.0596,0.6590,0.002660,0.0546,0.139,108.296,audio_features,29d0nY7TzCoi22XBqDQkiP,spotify:track:29d0nY7TzCoi22XBqDQkiP,https://api.spotify.com/v1/tracks/29d0nY7TzCoi...,https://api.spotify.com/v1/audio-analysis/29d0...,300840,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Fading,Rod Wave,0.674,0.656,11,-7.506,1,0.0625,0.3240,0.000000,0.1070,0.380,159.933,audio_features,0Ixokfhum2t1dXWg54hjPn,spotify:track:0Ixokfhum2t1dXWg54hjPn,https://api.spotify.com/v1/tracks/0Ixokfhum2t1...,https://api.spotify.com/v1/audio-analysis/0Ixo...,163500,4
96,Church Girl,Beyonce,0.792,0.919,2,-5.688,1,0.2760,0.0381,0.000005,0.3680,0.220,92.028,audio_features,2mqTtvbKxH7SoEQ2oGAnsA,spotify:track:2mqTtvbKxH7SoEQ2oGAnsA,https://api.spotify.com/v1/tracks/2mqTtvbKxH7S...,https://api.spotify.com/v1/audio-analysis/2mqT...,224473,4
97,Un Ratito,Bad Bunny,0.787,0.546,0,-7.094,0,0.0532,0.3050,0.000046,0.1220,0.222,93.050,audio_features,5CzixCxDkRXX9mScCmah8O,spotify:track:5CzixCxDkRXX9mScCmah8O,https://api.spotify.com/v1/tracks/5CzixCxDkRXX...,https://api.spotify.com/v1/audio-analysis/5Czi...,176936,4
98,La Corriente,Bad Bunny & Tony Dize,0.663,0.786,11,-3.510,0,0.1950,0.2290,0.000000,0.2190,0.579,196.120,audio_features,1797zYiX4cKosMH836X9Gt,spotify:track:1797zYiX4cKosMH836X9Gt,https://api.spotify.com/v1/tracks/1797zYiX4cKo...,https://api.spotify.com/v1/audio-analysis/1797...,198367,4


In [76]:
nothot=nothot.reset_index(drop=True)

In [77]:
nothot_all=pd.concat([nothot, feat_df],axis=1)

In [78]:
nothot_all

,song,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Easy On Me,Adele,0gplL1WMoJ6iYaPgMCL0gX,0.604,0.366,5,-7.519,1,0.0282,0.5780,...,0.1330,0.130,141.981,audio_features,0gplL1WMoJ6iYaPgMCL0gX,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,https://api.spotify.com/v1/tracks/0gplL1WMoJ6i...,https://api.spotify.com/v1/audio-analysis/0gpl...,224695,4
1,Industry Baby,Lil Nas X & Jack Harlow,27NovPIUIRrOZoCHxABJwK,0.736,0.704,3,-7.409,0,0.0615,0.0203,...,0.0501,0.894,149.995,audio_features,27NovPIUIRrOZoCHxABJwK,spotify:track:27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,212000,4
2,Fancy Like,Walker Hayes,58UKC45GPNTflCN6nwCUeF,0.647,0.765,1,-6.459,1,0.0600,0.1110,...,0.3150,0.855,79.994,audio_features,58UKC45GPNTflCN6nwCUeF,spotify:track:58UKC45GPNTflCN6nwCUeF,https://api.spotify.com/v1/tracks/58UKC45GPNTf...,https://api.spotify.com/v1/audio-analysis/58UK...,161853,4
3,Bad Habits,Ed Sheeran,3rmo8F54jFF8OgYsqTxm5d,0.807,0.893,11,-3.745,0,0.0347,0.0451,...,0.3660,0.537,126.011,audio_features,3rmo8F54jFF8OgYsqTxm5d,spotify:track:3rmo8F54jFF8OgYsqTxm5d,https://api.spotify.com/v1/tracks/3rmo8F54jFF8...,https://api.spotify.com/v1/audio-analysis/3rmo...,230747,4
4,Need To Know,Doja Cat,3Vi5XqYrmQgOYBajMWSvCi,0.664,0.609,1,-6.509,1,0.0707,0.3040,...,0.0926,0.194,130.041,audio_features,3Vi5XqYrmQgOYBajMWSvCi,spotify:track:3Vi5XqYrmQgOYBajMWSvCi,https://api.spotify.com/v1/tracks/3Vi5XqYrmQgO...,https://api.spotify.com/v1/audio-analysis/3Vi5...,210560,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2434,Why You Always Hatin?,YG Featuring Drake & Kamaiyah,4ezAjoV01phDz1SqO6SzEU,0.925,0.355,0,-9.309,0,0.5500,0.0413,...,0.1410,0.383,92.988,audio_features,4ezAjoV01phDz1SqO6SzEU,spotify:track:4ezAjoV01phDz1SqO6SzEU,https://api.spotify.com/v1/tracks/4ezAjoV01phD...,https://api.spotify.com/v1/audio-analysis/4ezA...,196587,4
2435,CRZY,Kehlani,6Dwtha2FtZFoMEBh5GR2sq,0.827,0.702,6,-7.264,1,0.2670,0.2260,...,0.1580,0.694,130.020,audio_features,6Dwtha2FtZFoMEBh5GR2sq,spotify:track:6Dwtha2FtZFoMEBh5GR2sq,https://api.spotify.com/v1/tracks/6Dwtha2FtZFo...,https://api.spotify.com/v1/audio-analysis/6Dwt...,191720,4
2436,Lockjaw,French Montana Featuring Kodak Black,7iaw359G2XT14uTfV9feip,0.615,0.648,5,-3.792,0,0.2200,0.0411,...,0.2770,0.260,169.912,audio_features,7iaw359G2XT14uTfV9feip,spotify:track:7iaw359G2XT14uTfV9feip,https://api.spotify.com/v1/tracks/7iaw359G2XT1...,https://api.spotify.com/v1/audio-analysis/7iaw...,223147,4
2437,For Free,DJ Khaled Featuring Drake,5TqBAOIBe0bQA0spNdcG3k,0.916,0.571,10,-8.241,0,0.2000,0.0173,...,0.1530,0.498,103.521,audio_features,5TqBAOIBe0bQA0spNdcG3k,spotify:track:5TqBAOIBe0bQA0spNdcG3k,https://api.spotify.com/v1/tracks/5TqBAOIBe0bQ...,https://api.spotify.com/v1/audio-analysis/5TqB...,183695,4


In [79]:
nothot_all=nothot_all.drop(['id'],axis=1)
hot_all=hot_all.drop(['id'],axis=1)

In [80]:
all=pd.concat([hot_all,nothot_all],axis=0)
all=all.reset_index(drop=True)

In [81]:
all['hot']='Yes'

In [82]:
all['hot'][100:2539]='No'

C:\Users\duyadong\AppData\Local\Temp\ipykernel_8852\1325886817.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all['hot'][100:2539]='No'


In [84]:
all.to_csv('all.csv')